In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import ticker
import awkward as ak
import uproot 
import mplhep as hep
import hist
import os
import time
from numba import njit
from scipy.optimize import curve_fit
from scipy.optimize import linear_sum_assignment
import tqdm



from UtilityAndConst import *
from Functions import *

dir = "/Users/francescolarovere/Documents/Tesi-Triennale/Dati"
os.chdir(dir)

hep.style.use("CMS")
np.set_printoptions(threshold=np.inf)

In [2]:
LS133 = "./ntupleStubsBmtfGmt_LS133.root"
gmt_awk_LS133 = OpenRootFile(LS133, gmt_features)
bmtf_awk_LS133 = OpenRootFile(LS133, bmtf_features)

In [3]:
Gmt_Phi_LS133 = gmt_awk_LS133.gmtPhi
Bmtf_Phi_LS133 = bmtf_awk_LS133.bmtfPhi

Gmt_Eta_LS133 = gmt_awk_LS133.gmtEta
Bmtf_Eta_LS133 = bmtf_awk_LS133.bmtfEta

nGmtMuons_LS133 = gmt_awk_LS133.nGmtMuons
nBmtfMuons_LS133 = bmtf_awk_LS133.nBmtfMuons


nBmtfMuons_LS133[192]


2

In [4]:
arr = [1, 2, 3, 4, 4]

min = np.min(arr)

arr = [x for x in arr if x!= min]
    

print(arr)

[2, 3, 4, 4]


In [5]:
def Delta(BMTF, GMT):
  return BMTF - GMT

def DeltaR(BmtfPhi, GmtPhi, BmtfEta, GmtEta):
  return np.sqrt((BmtfPhi - GmtPhi)**2 + (BmtfEta - GmtEta)**2)

In [18]:
i = 192
#192
#530


DeltaRDistBMTF = []
IndexMinBMTF = []
DeltaRDistGMT = []
IndexMinGMT = []

ArrBmtfPhi, ArrBmtfEta = ak.to_numpy(Bmtf_Phi_LS133[i]), ak.to_numpy(Bmtf_Eta_LS133[i])
ArrGmtPhi, ArrGmtEta = ak.to_numpy(Gmt_Phi_LS133[i]), ak.to_numpy(Gmt_Eta_LS133[i])

print("InizialeBMTF: ", ArrBmtfEta, " ", ArrBmtfPhi)
print("InizialeGMT: ", ArrGmtEta, " ", ArrGmtPhi)


for j in range(nBmtfMuons_LS133[i]):

                
  #Calcolo il DeltaR
  DeltaRTemp = DeltaR(ArrBmtfPhi[j], ArrGmtPhi[:], ArrBmtfEta[j], ArrGmtEta[:])

  minDeltaR = np.min(DeltaRTemp)
  indexTempGMT = np.argmin(DeltaRTemp)
  IndexMinGMT.append(indexTempGMT)

  if minDeltaR < 0.4:

    if IndexMinGMT[j] in IndexMinGMT[:j]:
      IndexMinGMT.pop(j)
      DeltaRTemp = [x for x in DeltaRTemp if x != minDeltaR]

      minDeltaR = np.min(DeltaRTemp)
      DeltaRDistBMTF.append(minDeltaR)
      IndexMinGMT.append(indexTempGMT)

    else:
      DeltaRDistBMTF.append(minDeltaR)


  print("Minimo: ", DeltaRTemp)
  print("Indice del Minimo", IndexMinGMT)

  


InizialeBMTF:  [0.51112497 0.52199996]   [-2.8907015 -2.8907015]
InizialeGMT:  [0.52199996]   [-2.8907015]
Minimo:  [0.01087499]
Indice del Minimo [0]


ValueError: zero-size array to reduction operation minimum which has no identity

## CODICE VERO!

In [13]:
MuonsProperties = []

use_Bmtf = True


for i in tqdm.tqdm(range(50000)):

  nBmtfM, nGmtM = nBmtfMuons_LS133[i], nGmtMuons_LS133[i]
  DeltaRDistBMTF = []
  IndexMinBMTF = []
  DeltaRDistGMT = []
  IndexMinGMT = []

  if nGmtM == 0 and nBmtfM == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = []
  
  elif nBmtfM == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = [i for i in range(nGmtM)]

  elif nGmtM == 0:
    unmatched_Bmtf = [i for i in range(nBmtfM)]
    unmatched_Gmt = []

  else:
    #Fare in questo modo ottimizza enormemente il processo!
    ArrBmtfPhi, ArrBmtfEta = ak.to_numpy(Bmtf_Phi_LS133[i]), ak.to_numpy(Bmtf_Eta_LS133[i])
    ArrGmtPhi, ArrGmtEta = ak.to_numpy(Gmt_Phi_LS133[i]), ak.to_numpy(Gmt_Eta_LS133[i])

    #Primo metodo 
    for j in range(nBmtfM):

      #Calcolo il DeltaR
      DeltaRTemp = DeltaR(ArrBmtfPhi[j], ArrGmtPhi[:], ArrBmtfEta[j], ArrGmtEta[:])

      #Attento a come sono definiti IndexMin nei vari casi! Nel primo caso è confronto con tutti i muoni del GMT!
      minDeltaR = np.min(DeltaRTemp)
      indexTempGMT = np.argmin(DeltaRTemp)
      IndexMinGMT.append(indexTempGMT)

      if minDeltaR < 0.4:
        
        #Controllo che non vengano assegnati gli stessi indici dei minimi a muoni diversi
        if IndexMinGMT[j] in IndexMinGMT[:j]:
          IndexMinGMT.pop(j)
          DeltaRTemp = [x for x in DeltaRTemp if x != minDeltaR]

          minDeltaR = np.min(DeltaRTemp)
          DeltaRDistBMTF.append(minDeltaR)
          IndexMinGMT.append(indexTempGMT)

        else:
          DeltaRDistBMTF.append(minDeltaR)


        DeltaPhi_BMTF = Delta(ArrBmtfPhi[j], ArrGmtPhi[IndexMinGMT[j]])
        DeltaEta_BMTF = Delta(ArrBmtfEta[j], ArrGmtEta[IndexMinGMT[j]])
        
        

      else:
        #Se DeltaR > 0.4 (primo metodo) non c'è match con i muoni del GMT
        unmatched_Gmt = [i for i in range(nGmtM)]
  
    
      matched_Bmtf_Index = [i for i in IndexMinGMT if IndexMinGMT]
      unmatched_Bmtf = [i for i in range(nBmtfM) if i not in matched_Bmtf_Index]
      

    #Secondo metodo 
    for k in range(nGmtM):
      DeltaRTemp = DeltaR(ArrBmtfPhi[:], ArrGmtPhi[k], ArrBmtfEta[:], ArrGmtEta[k])

      minDeltaR = np.min(DeltaRTemp)
      indexTempBMTF = np.argmin(DeltaRTemp)
      IndexMinBMTF.append(indexTempBMTF)

      if minDeltaR < 0.4:
        
        if IndexMinBMTF[k] in IndexMinBMTF[:k]:

          IndexMinBMTF.pop(k)
          DeltaRTemp = [y for y in DeltaRTemp if y != minDeltaR]

          minDeltaR = np.min(DeltaRTemp)
          DeltaRDistGMT.append(minDeltaR)
          IndexMinBMTF.append(indexTempBMTF)
        
        else:
          DeltaRDistGMT.append(minDeltaR)

          
        DeltaPhi_GMT = Delta(ArrBmtfPhi[IndexMinBMTF[k]], ArrGmtPhi[k])
        DeltaEta_GMT = Delta(ArrBmtfEta[IndexMinBMTF[k]], ArrGmtEta[k])
        

      
      else:
        unmatched_Bmtf = [i for i in range(nBmtfM)]

      matched_Gmt_Index = [k for k in IndexMinBMTF if IndexMinBMTF]
      unmatched_Gmt = [k for k in range(nGmtM) if k not in matched_Gmt_Index]



  Events = {
    'unmatched_Gmt_Events' : unmatched_Gmt,
    'unmatched_Bmtf_Events': unmatched_Bmtf, 
  }

  MuonsProperties.append(Events)


  0%|          | 192/50000 [00:00<00:05, 9410.42it/s]


ValueError: zero-size array to reduction operation minimum which has no identity

In [8]:
MuonsProperties = ak.Array(MuonsProperties)
print(ak.sum(ak.num(MuonsProperties.unmatched_Bmtf_Events)))
print(ak.sum(ak.num(MuonsProperties.unmatched_Gmt_Events)))

554
94


## PRIMA DELLA MODIFICA

In [16]:
MuonsProperties = []

use_Bmtf = True


for i in tqdm.tqdm(range(50000)):

  
  DeltaRDistBMTF = []
  IndexMinBMTF = []
  DeltaRDistGMT = []
  IndexMinGMT = []

  if nGmtMuons_LS133[i] == 0 and nBmtfMuons_LS133[i] == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = []
  
  elif nBmtfMuons_LS133[i] == 0:
    unmatched_Bmtf = []
    unmatched_Gmt = [i for i in range(nGmtMuons_LS133[i])]

  elif nGmtMuons_LS133[i] == 0:
    unmatched_Bmtf = [i for i in range(nBmtfMuons_LS133[i])]
    unmatched_Gmt = []

  else:
    #Fare in questo modo ottimizza enormemente il processo!
    ArrBmtfPhi, ArrBmtfEta = ak.to_numpy(Bmtf_Phi_LS133[i]), ak.to_numpy(Bmtf_Eta_LS133[i])
    ArrGmtPhi, ArrGmtEta = ak.to_numpy(Gmt_Phi_LS133[i]), ak.to_numpy(Gmt_Eta_LS133[i])

    #Primo metodo 
    for j in range(nBmtfMuons_LS133[i]):

      #Calcolo il DeltaR
      DeltaRTemp = DeltaR(ArrBmtfPhi[j], ArrGmtPhi[:], ArrBmtfEta[j], ArrGmtEta[:])

      minDeltaR = np.min(DeltaRTemp)

      if minDeltaR < 0.4:

        #Attento a come sono definiti IndexMin nei vari casi! Nel primo caso è confronto con tutti i muoni del GMT!
        indexTempGMT = np.argmin(DeltaRTemp)
        DeltaRDistBMTF.append(minDeltaR)
        IndexMinGMT.append(indexTempGMT)
        DeltaPhi_BMTF = Delta(ArrBmtfPhi[j], ArrGmtPhi[indexTempGMT])
        DeltaEta_BMTF = Delta(ArrBmtfEta[j], ArrGmtEta[indexTempGMT])
        #ArrGmtPhi, ArrGmtEta = np.delete(ArrGmtPhi, indexTempGMT), np.delete(ArrGmtEta, indexTempGMT)

      else:
        #Se DeltaR > 0.4 (primo metodo) non c'è match con i muoni del GMT
        unmatched_Gmt = [i for i in range(nGmtMuons_LS133[i])]
  
    
      matched_Bmtf_Index = [i for i in IndexMinGMT if IndexMinGMT]
      unmatched_Bmtf = [i for i in range(nBmtfMuons_LS133[i]) if i not in matched_Bmtf_Index]


      #Tolgo gli elementi del GMT (quelli che vengono confrontati) già matchati
      

    
    #Riassegno per ripristinare il valore iniziale in ogni BX
    ArrBmtfPhi, ArrBmtfEta = ak.to_numpy(Bmtf_Phi_LS133[i]), ak.to_numpy(Bmtf_Eta_LS133[i])
    ArrGmtPhi, ArrGmtEta = ak.to_numpy(Gmt_Phi_LS133[i]), ak.to_numpy(Gmt_Eta_LS133[i])

    #Secondo metodo 
    for k in range(nGmtMuons_LS133[i]):
      DeltaRTemp = DeltaR(ArrBmtfPhi[:], ArrGmtPhi[k], ArrBmtfEta[:], ArrGmtEta[k])

      minDeltaR = np.min(DeltaRTemp)

      if minDeltaR < 0.4:
  
        indexTempBMTF = np.argmin(DeltaRTemp)
        DeltaRDistGMT.append(minDeltaR)
        IndexMinBMTF.append(indexTempBMTF)
        DeltaPhi_GMT = Delta(ArrBmtfPhi[indexTempBMTF], ArrGmtPhi[k])
        DeltaEta_GMT = Delta(ArrBmtfEta[indexTempBMTF], ArrGmtEta[k])
        #ArrBmtfPhi, ArrBmtfEta = np.delete(ArrBmtfPhi, indexTempGMT), np.delete(ArrBmtfEta, indexTempGMT)
      
      else:
        unmatched_Bmtf = [i for i in range(nBmtfMuons_LS133[i])]

      matched_Gmt_Index = [k for k in IndexMinBMTF if IndexMinBMTF]
      unmatched_Gmt = [k for k in range(nGmtMuons_LS133[i]) if k not in matched_Gmt_Index]

      #Tolgo il valore 




  Events = {
    'unmatched_Gmt_Events' : unmatched_Gmt,
    'unmatched_Bmtf_Events': unmatched_Bmtf, 
  }

  MuonsProperties.append(Events)

100%|██████████| 50000/50000 [00:05<00:00, 8534.32it/s]


In [17]:
MuonsProperties = ak.Array(MuonsProperties)
print(ak.sum(ak.num(MuonsProperties.unmatched_Bmtf_Events)))
print(ak.sum(ak.num(MuonsProperties.unmatched_Gmt_Events)))

554
94
